<a href="https://colab.research.google.com/github/ClaudiaMarano/Anomaly-Detection-and-Prediction/blob/main/network_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physical Anomaly Detection

## Import Librerie

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.callbacks import Callback
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU
from keras.layers import Dropout

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


## Caricamento e Preprocessing
Il dataset viene suddiviso in intervalli di un minuto in modo da addestrare la rete su intervalli di campioni senza anomalie, in modo da poter riconoscere in fase di test quando un intervallo contiene invece un'anomalia.


### Caricamento e Divisione Dataset

In [15]:
def load_and_split_data(path):
    # Caricamento del dataset
    df = pd.read_csv(path, encoding="utf-8")

    # Assicuriamoci che 'Time' sia in formato datetime
    df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

    # Rimuovi righe con valori di data non validi
    df = df.dropna(subset=['Time'])

    # Ordinare i dati per timestamp
    df_normal = df.sort_values(by='Time')

    # Divisione in intervalli di un minuto
    segments = []
    window_duration = pd.Timedelta(milliseconds=100)
    start_time = df_normal['Time'].iloc[0]

    print("Start Time: ", df_normal['Time'].iloc[0])
    print("Finish Time: ", df_normal['Time'].iloc[-1])

    while start_time < df_normal['Time'].iloc[-1]:
        end_time = start_time + window_duration
        segment = df_normal[(df_normal['Time'] >= start_time) & (df_normal['Time'] < end_time)]
        if len(segment) > 0:
            segments.append(segment.drop(columns=['Time', 'label', 'label_n', 'modbus_response']).reset_index(drop=True).values)
        start_time = end_time

    print(type(segments[0]))

    # Filtra i segmenti per ottenere solo quelli esattamente di 200 righe
    valid_segments = []
    for segment in segments:
        if len(segment) > 200:
            # Mantieni solo le prime 200 righe
            valid_segments.append(segment[:200])
        elif len(segment) == 200:
            # Segmento già valido
            valid_segments.append(segment)

    # print(f"Number of valid segments: {len(valid_segments)}")

    return valid_segments

### Preprocessing Dataset

In [44]:
# 2.
def preprocessing(segments):
    """
    1) Accorpo tutti i segmenti in un unico numpy array per poter applicare la normalizzazione.
    2) Normalizzo.
    3) Divido di nuovo i segmenti normalizzati nel numero originario dei segmenti in input.
    4) Ritorno i segmenti normalizzati e lo scaler utilizzato.
    :param segments:
    :return:
    """
    print(f"Tipologia: {type(segments[0])}")
    # Preprocessing delle feature
    # Separiamo colonne categoriali e numeriche

    categorical_columns = [0, 1, 2, 3, 6, 7, 9]
    numerical_columns = [4, 5, 8, 10, 11]

    # 1)
    segments_array = np.vstack(segments)

    print(f"Array Unico: {segments_array}")
    print(f"Tipologia: {type(segments_array)}")
    print(f"Lunghezza: {len(segments_array)}")
    print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")

    # 2)
    # Configura il preprocessore con categorie globali e scaling numerico
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(categories='auto', handle_unknown='ignore'), categorical_columns),
            ('num', MinMaxScaler(), numerical_columns)
        ]
    )

    segments_scaled = preprocessor.fit_transform(segments_array)

    print(f"Segmenti Normalizzati: {segments_scaled[0:1]}")
    print(f"Tipologia: {type(segments_scaled)}")
    print(f"Lunghezza: {segments_scaled.shape}")
    print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")

    # 3)
    segments_scaled_split = np.array_split(segments_scaled.toarray(), len(segments))

    print(f"Segmenti Normalizzati e Ricostruiti: {segments_scaled_split}")
    print(f'Tipologia: {type(segments_scaled_split)}')
    print(f"Lunghezza: {len(segments_scaled_split)}")
    print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")

    # 4)
    return segments_scaled_split, preprocessor

## Funzione per il Training

In [48]:
# 3.
def building_and_training(segments_scaled_split):
    # Callback per visualizzare statistiche al termine dell'addestramento
    class TrainingSummary(Callback):
        def on_train_end(self, logs=None):
            print("\n--- Statistiche Finali ---")
            print(f"Loss finale su training set: {logs['loss']:.4f}")
            if 'val_loss' in logs:
                print(f"Loss finale su validation set: {logs['val_loss']:.4f}")

    # Definisco l'input shape
    input_shape = segments_scaled_split[0].shape  # Forma di un segmento

    # Definisco la struttura dell'autoencoder
    input_layer = Input(shape=input_shape)
    x = Flatten()(input_layer)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 10%
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 10%
    encoded = Dense(64)(x)
    encoded = LeakyReLU(alpha=0.1)(encoded)
    x = Dense(128)(encoded)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 10%
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 10%
    x = Dense(np.prod(input_shape), activation="sigmoid")(x)
    x = BatchNormalization()(x)
    decoded = Reshape(input_shape)(x)

    # Creo il Modello
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    # Addestramento dell'autoencoder sui dati "normali", senza anomalie
    history = autoencoder.fit(
        np.array(segments_scaled_split), np.array(segments_scaled_split),
        epochs=100, batch_size=16, shuffle=True, validation_split=0.1,
        callbacks=[TrainingSummary(), early_stopping]
    )

    # Stampo statistiche finali direttamente dal dizionario `history.history`

    print("\n--- Risultati Finali ---")
    print(f"Training Loss: {history.history['loss'][-1]:.4f}")
    print(f"Validation Loss: {history.history['val_loss'][-1]:.4f}")

    return autoencoder


## Calcolo dell'Errore di Ricostruzione

## Test del modello

## Esecuzione del processo

In [35]:
segments = load_and_split_data("normal_reduced_0005.csv")
print("Numero di segmenti:", len(segments))
print(segments[0].shape)

Start Time:  2021-04-09 11:30:52.716203
Finish Time:  2021-04-09 11:30:54.462284
<class 'numpy.ndarray'>
Numero di segmenti: 17
(200, 12)


In [ ]:
processed_segments, preprocessor = preprocessing(segments)

In [ ]:
autoencoder = building_and_training(processed_segments)

In [ ]:
threshols = get_rebuilding_error(autoencoder, segments)

In [ ]:
testing("attack_1.csv", scaler, autoencoder, threshols)